In [17]:
from gmplot import gmplot
%matplotlib inline

# Place map
gmap = gmplot.GoogleMapPlotter(37.766956, -122.438481, 13)

# Polygon
golden_gate_park_lats, golden_gate_park_lons = zip(*[
    (37.771269, -122.511015),
    (37.773495, -122.464830),
    (37.774797, -122.454538),
    (37.771988, -122.454018),
    (37.773646, -122.440979),
    (37.772742, -122.440797),
    (37.771096, -122.453889),
    (37.768669, -122.453518),
    (37.766227, -122.460213),
    (37.764028, -122.510347),
    (37.771269, -122.511015)
    ])
gmap.plot(golden_gate_park_lats, golden_gate_park_lons, 'cornflowerblue', edge_width=10)

# Scatter points
top_attraction_lats, top_attraction_lons = zip(*[
    (37.769901, -122.498331),
    (37.768645, -122.475328),
    (37.771478, -122.468677),
    (37.769867, -122.466102),
    (37.767187, -122.467496),
    (37.770104, -122.470436)
    ])
gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False)

# Marker
hidden_gem_lat, hidden_gem_lon = 37.770776, -122.461689
gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')

# Draw
gmap.draw("my_map.html")

In [24]:
def build_query(outFormat = 'text', starttime = '2016-02-01', endtime = '2016-03-01', minmagnitude = 5.0):
    '''
    Funciton to build the url for query the data. Details can be found at USGS api documentation:
    http://earthquake.usgs.gov/fdsnws/event/1/
    
    Note: you can add more parameters, but I just need time range and minmum magnitude to start with. 
    '''
    base = 'http://earthquake.usgs.gov/fdsnws/event/1/query?'
    url = base + 'format=' + outFormat + '&starttime=' + starttime + '&endtime=' + endtime + '&minmagnitude=' + str(minmagnitude)
    return url

In [26]:
def parse_result(inputText):
    '''
    Function to parse the requested earthquake events data from USGS, and save it into numpy array
    '''
    event_id = []
    origin_time = []
    evla = []
    evlo = []
    evdp = []
    mag = []
    mag_type = []
    EventLocationName  = []
    for i, item in enumerate(inputText.split('\n')[0:-1]):
        if i < 1:
            # skip the header
            continue
            
        try:
            splited = item.split('|')
            event_id.append(splited[0])  
            origin_time.append(splited[1])
            evla.append(splited[2])
            evlo.append(splited[3])
            evdp.append(splited[4])
            mag.append(splited[10])
            mag_type.append(splited[9])
            EventLocationName.append(splited[-1])
        except:
            # just in case there are some wrong data in the catlog
            print (item)
            print ('Skip wrong data or there is something wrong') 
        
    return np.c_[event_id, origin_time, evla, evlo, mag, mag_type, EventLocationName]

In [29]:
import urllib.request
from bs4 import BeautifulSoup
import numpy as np

In [30]:
url = build_query(outFormat = 'text', starttime = '2010-01-01', endtime = '2016-01-01', minmagnitude = 5.0)
print (url)

# get the earthquake data from USGS and parse them into a numpy array
with urllib.request.urlopen(url) as url:
    s = url.read()
r = s
soup = BeautifulSoup(r, "lxml")
events_mat = parse_result(soup.text)

http://earthquake.usgs.gov/fdsnws/event/1/query?format=text&starttime=2010-01-01&endtime=2016-01-01&minmagnitude=5.0


In [31]:
lats = [float(item[2]) for item in events_mat]
lons = [float(item[3]) for item in events_mat]

In [33]:
# declare the center of the map, and how much we want the map zoomed in
gmap = gmplot.GoogleMapPlotter(0, 0, 2)
# plot heatmap
gmap.heatmap(lats, lons)
# save it to html
gmap.draw("Earthquake_heatmap.html")

#### Minneapolis 311 2017

In [36]:
import pandas as pd
df = pd.read_csv('./Data/Public_311_2017.csv')

In [48]:
gmap = gmplot.GoogleMapPlotter.from_geocode("Minneapolis")
gmap.heatmap(df['Y'], df['X'])
gmap.draw("./Output/311_2017.html")

#### Minneapolis Snow Emergency Jane Tows 2017

In [53]:
import json
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Snow_Emergency_Jane_Tows_2017/\
FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
with urllib.request.urlopen(url) as url:
    s = url.read()

In [58]:
#from pandas.io.json import json_normalize
df = pd.io.json.json_normalize(json.loads(s)['features'])

In [63]:
gmap = gmplot.GoogleMapPlotter.from_geocode("Minneapolis",12)
gmap.heatmap(df['geometry.y'], df['geometry.x'])
gmap.draw("./Output/snow_plow_2017.html")

#### Minneapolis Police Incidents 2016

In [68]:
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/\
rest/services/PoliceOffense2016/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
with urllib.request.urlopen(url) as url:
    s = url.read()
df = pd.io.json.json_normalize(json.loads(s)['features'])

In [69]:
gmap = gmplot.GoogleMapPlotter.from_geocode("Minneapolis",12)
gmap.heatmap(df['geometry.y'], df['geometry.x'])
gmap.draw("./Output/police_incidents_2016.html")

#### Chicago Crimes since 2001

In [ ]:
df = pd.read_csv('Crimes_-_2001_to_present.csv')

In [ ]:
df.head()